<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/tokenizer_test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [1]:
import numpy as np


# **Data**

In [47]:
filename = '/content/drive/MyDrive/Kiswahili_Dataset/Kiswdata1.txt'
data = np.loadtxt(filename)
print(data)

[]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: loadtxt: Empty input file: "/content/drive/MyDrive/Kiswahili_Dataset/Kiswdata1.txt"
  


In [45]:
file1 = open("/content/drive/MyDrive/Kiswahili_Dataset/Kiswdata3.txt","a")
print(file1)

<_io.TextIOWrapper name='/content/drive/MyDrive/Kiswahili_Dataset/Kiswdata3.txt' mode='a' encoding='UTF-8'>


In [39]:
file2 = open(r"/content/drive/MyDrive/Kiswahili_Dataset/Kiswdata2.txt","w+")
print(file2)

<_io.TextIOWrapper name='/content/drive/MyDrive/Kiswahili_Dataset/Kiswdata2.txt' mode='w+' encoding='UTF-8'>


# **Tokenizer**

In [12]:
import os

In [ ]:
os.mkdir('/content/drive/MyDrive/Kiswahili_Dataset2')

In [48]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(data):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'/content/drive/MyDrive/Kiswahili_Dataset2_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we may have leftover samples, we save those now too
with open(f'/content/drive/MyDrive/Kiswahili_Dataset2_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

0it [00:00, ?it/s]

## **Training**

In [41]:
from pathlib import Path
paths = [str(x) for x in Path('/content/drive/MyDrive/Kiswahili_Dataset2').glob('**/*.txt')]
paths[:5]

[]

In [36]:


len(paths)



0

In [20]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 4.8 MB/s 


In [21]:

from tokenizers import BertWordPieceTokenizer

# initialize
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=False
)
# and train
tokenizer.train(files=paths, vocab_size=30_000, min_frequency=2,
                limit_alphabet=1000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

# **Save Tokenizer**

In [22]:
os.mkdir('/content/drive/MyDrive/Kiswahili_tokenizer')

tokenizer.save_model('/content/drive/MyDrive/Kiswahili_tokenizer', 'kisw-tok')

['/content/drive/MyDrive/Kiswahili_tokenizer/kisw-tok-vocab.txt']

# **Using the Tokenizer**

In [24]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.4 MB/s 
     |████████████████████████████████| 163 kB 55.3 MB/s 


In [26]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Kiswahili_tokenizer/kisw-tok-vocab.txt')


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1682: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
tokenizer('Habari yako?')  # how are you?

{'input_ids': [2, 1, 1, 1, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

# **Testing Something**

In [50]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 48.8 MB/s 
     |████████████████████████████████| 95 kB 4.9 MB/s 
     |████████████████████████████████| 115 kB 50.4 MB/s 
     |████████████████████████████████| 127 kB 48.4 MB/s 
     |████████████████████████████████| 115 kB 46.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [51]:
#Libraries
import pandas as pd
import numpy as np
import datasets

In [52]:
dv = datasets.load_dataset(
    '/content/drive/MyDrive/Kiswahili_Dataset', 'train',
    streaming=True
)

dv

{'train': <datasets.iterable_dataset.IterableDataset at 0x7fd8c36d4fd0>}

In [53]:
dv['train']

In [54]:
count = 0
for row in dv['train']:
    print(row)
    count += 1
    if count == 3: break

{'text': 'KNEC STUDY MATERIALS, REVISION KITS AND PAST PAPERSSTUDY FOR FREEKenya Certificate of Secondary Education Kiswahili Karatasi ya 3SEHEMU A : TAMTHILIAP.Kea:Kigogo1. Lazima(a) “Aikose? Asiya ni mwanamke halisi bwana! Amemweka Bi Husda hapa.” (i) Eleza muktadha wa dondoo hili. (alama 10)(ii) Eleza vipengele viwili vya kimtindo alivyotumia msemaji kufanikisha uwasilishaji wa ujumbe.(alama 2)(iii) Fafanua sifa nne za anayeambiwa maneno haya.(alama 4)(b) “Kuzorota kwa maadili kunachangia kuzorota kwa mahusiano ya kijamii.” Thibitisha kauli hii kwa kurejelea tamthilia ya Kigogo. (alama 10)SEHEMU B: RIWAYAA. Matei: Chozi la Heri Jibu swali la 2 au la 3.2. Jadili nafasi ya mwanamke katika kuendeleza ukiukaji wa haki za kibinadamu katika riwaya ya Chozi la Heri. {alama 20)3. (a) Changanua mtindo katika dondoo lifuatalo:“Una babati sana mweozangu,” alisema Kairi siku moja, ‘Wewe hujapitia tuliyopitia. Tulitendwa ya kutendwa. Siku hiyo tulipofurushwa kwetu kama takataka tulitanga na njia

In [64]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(row):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'/content/drive/MyDrive/Kiswahili_Dataset2_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we may have leftover samples, we save those now too
with open(f'/content/drive/MyDrive/Kiswahili_Dataset2_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/1 [00:00<?, ?it/s]

TypeError: ignored

In [61]:
from pathlib import Path
paths = [str(x) for x in Path('/content/drive/MyDrive/Kiswahili_Dataset2').glob('**/*.txt')]
paths[:5]

[]

In [62]:
len(paths)

0